In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import drive
filename = '/content/datasample1.xlsx'

In [ ]:
df = pd.read_excel(filename)
df.head()

In [ ]:
df['시간'] = df['시간']-1
df.head()

In [ ]:
df['dateInt']= df['연'].astype(str) + df['월'].astype(str).str.zfill(2)+df['일'].astype(str).str.zfill(2)+df['시간'].astype(str).str.zfill(2)
df['dateInt']

In [ ]:
df['Date'] = pd.to_datetime(df['dateInt'], format="%Y%m%d%H")
df['Date']

In [ ]:
df.insert(4, '일자', value = df['Date'])
df.head()

In [ ]:
df =df.drop(['연','월','일','시간','dateInt','Date'], axis = 1)
df.head()

In [ ]:
df.sort_index(ascending=False).reset_index(drop=True)

scaler = MinMaxScaler()
scale_cols = ['유입량', '데이터집단 1', 'Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 10','Unnamed: 11','Unnamed: 12','데이터집단 2','Unnamed: 14','Unnamed: 15','Unnamed: 16','Unnamed: 17','Unnamed: 18','Unnamed: 19','데이터집단 3','Unnamed: 21','Unnamed: 22','Unnamed: 23','Unnamed: 24','Unnamed: 25','Unnamed: 26','데이터 집단 4','Unnamed: 28','Unnamed: 29','Unnamed: 30','Unnamed: 31','Unnamed: 32','Unnamed: 33','데이터 집단 5','Unnamed: 35','Unnamed: 36','Unnamed: 37','Unnamed: 38','Unnamed: 39','Unnamed: 40','데이터 집단 6','Unnamed: 42','Unnamed: 43','Unnamed: 44','Unnamed: 45','Unnamed: 46','Unnamed: 47']
df_scaled = scaler.fit_transform(df[scale_cols])
df_scaled = pd.DataFrame(df_scaled)
df_scaled.columns = scale_cols

df_scaled

In [ ]:
plt.figure(figsize=(20, 10))
sns.lineplot(y=df_scaled['유입량'], x=df['일자'])
plt.xlabel('TIME')
plt.ylabel('AMOUNT')

In [ ]:
TEST_SIZE = 160
VAL_SIZE = 1118

train1 = df_scaled[:-TEST_SIZE]
test = df_scaled[-TEST_SIZE:]
#print(train1)
#print(test)

train = train1[:-VAL_SIZE]
#print(train)
validation = train1[-VAL_SIZE:]
#print(validation)

In [ ]:
def make_dataset(data, label, window_size=20):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size]))
    return np.array(feature_list), np.array(label_list)

In [ ]:
from sklearn.model_selection import train_test_split

feature_cols = ['데이터집단 1', 'Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 10','Unnamed: 11','Unnamed: 12','데이터집단 2','Unnamed: 14','Unnamed: 15','Unnamed: 16','Unnamed: 17','Unnamed: 18','Unnamed: 19','데이터집단 3','Unnamed: 21','Unnamed: 22','Unnamed: 23','Unnamed: 24','Unnamed: 25','Unnamed: 26','데이터 집단 4','Unnamed: 28','Unnamed: 29','Unnamed: 30','Unnamed: 31','Unnamed: 32','Unnamed: 33','데이터 집단 5','Unnamed: 35','Unnamed: 36','Unnamed: 37','Unnamed: 38','Unnamed: 39','Unnamed: 40','데이터 집단 6','Unnamed: 42','Unnamed: 43','Unnamed: 44','Unnamed: 45','Unnamed: 46','Unnamed: 47']
label_cols = ['유입량']

train_feature = train[feature_cols]
train_label = train[label_cols]

#train_label

In [ ]:
train_feature, train_label = make_dataset(train_feature, train_label, 20)

x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2)
x_train.shape, x_valid.shape

#train_feature

In [ ]:
val_feature = validation[feature_cols]
val_label = validation[label_cols]

val_feature.shape, val_label.shape

#print(val_feature)

In [ ]:
val_feature, val_label = make_dataset(val_feature, val_label, 20)
val_feature.shape, val_label.shape

#print(val_feature)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM

model = Sequential()
model.add(LSTM(20, input_shape=(train_feature.shape[1], train_feature.shape[2]), activation='hard_sigmoid', return_sequences=False))
model.add(Dense(1))

In [ ]:
import os

model.compile(loss='mean_squared_error', optimizer='adam')
early_stop = EarlyStopping(monitor='val_loss', patience=5)
model_path = 'model'
filename = os.path.join(model_path, 'tmp_checkpoint.h5')
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

history = model.fit(x_train, y_train, 
                                    epochs=100, 
                                    batch_size=32,
                                    validation_data=(x_valid, y_valid), 
                                    callbacks=[early_stop, checkpoint])

In [ ]:
model.load_weights(filename)
pred = model.predict(val_feature)

pred.shape

In [ ]:
from sklearn.metrics import mean_squared_error
RMSE = mean_squared_error(val_label,pred)**0.5
RMSE

In [ ]:
plt.figure(figsize=(12, 9))
plt.plot(val_label, label = 'actual val')
plt.plot(pred, label = 'prediction')
plt.legend()
plt.show()